# Using Snowflake Cortex ❄️🤖

Snowflake Cortex gives you instant access to industry-leading large language models (LLMs) trained by researchers at companies like Anthropic, Mistral, Reka, Meta, and Google, including Snowflake's own Arctic, an open enterprise-grade model.

In this notebook, we'll cover only selected Cortex functions used in this course. 

## Snowflake Cortex

Cortex can be used via a SQL query or the Python SDK.

In a nutshell, there are 3 major LLM functions that we'll also cover here:
1. `COMPLETE` function
2. Helper functions (`COUNT_TOKENS`, `SPLIT_TEXT_RECURSIVE_CHARACTER`)
3. Task-specific functions (`PARSE_DOCUMENT`, `SENTIMENT`, `TRANSLATE`)

## Prerequisite libraries
Install these packages: `snowflake` and `snowflake-ml-python`
- In the top menu bar, click on "Packages" 
- In "Anaconda Packages" tab find "Find Packages" text box and enter `snowflake` then `snowflake-ml-python`
- Wait a few moments for packages to install

## 1. `COMPLETE` function

Given a prompt, generates a response (completion) using your choice of supported language model.

Learn more about [COMPLETE](https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex)



### SQL

First, we'll start by using Cortex in a SQL cell.

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'claude-3-5-sonnet', 
    'What is Python?'
) as RESPONSE;

To use Cortex via a SQL query in Python, we'll start by getting an active Snowflake session. 

Next, we'll use the `snowflake.cortex.complete` SQL function in Python via `session.sql()`.

In [ ]:
# Get active Snowflake session

from snowflake.snowpark.context import get_active_session

session = get_active_session()

In [ ]:
session.sql("SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet', 'What is Python?') as RESPONSE;")

### Python

Cortex could also be run natively in Python through the `snowflake` Python library.

In [ ]:
from snowflake.cortex import complete

complete('claude-3-5-sonnet',"What is Python?")

## 2. Helper functions

Helper functions are purpose-built and managed functions that reduce cases of failures when running other LLM functions.

For example, by getting the count of tokens in an input prompt to ensure the call doesn’t exceed a model limit.

Learn more about [Cortex LLM functions](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions)

### `COUNT_TOKENS`

Given an input text, returns the token count based on the model or Cortex function specified.

Learn more about [COUNT_TOKEN](https://docs.snowflake.com/en/sql-reference/functions/count_tokens-snowflake-cortex)

In [ ]:
SELECT SNOWFLAKE.CORTEX.COUNT_TOKENS(
    'llama3-8b',
    'The snow goggles have been a great investment for winter sports'
) AS RESPONSE

### `SPLIT_TEXT_RECURSIVE_CHARACTER`

The function splits a string into shorter stings, recursively, for preprocessing text to be used with text embedding or search indexing functions. 

In short, it returns an array of text chunks.

Learn more about [SPLIT_TEXT_RECURSIVE_CHARACTER](https://docs.snowflake.com/en/sql-reference/functions/split_text_recursive_character-snowflake-cortex)

In [ ]:
SELECT SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
   'The snow goggles have been a great investment for winter sports',
   'none',
   15,
   10
) as RESPONSE;

## 3. Task-specific LLM functions

Learn more about [Task-specific functions](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions#task-specific-functions)

### `PARSE_DOCUMENTS`

Provides the ability to extract text, data, and layout elements from documents.

Learn more about [PARSE_DOCUMENTS](https://docs.snowflake.com/en/sql-reference/functions/parse_document-snowflake-cortex)

In [ ]:
-- Read single file
SELECT
  SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
    @avalanche_db.avalanche_schema.customer_reviews,
    'review-100.docx',
    {'mode': 'layout'}
  ) AS layout;

In [ ]:
-- Read multiple files into a table
WITH files AS (
  SELECT 
    REPLACE(REGEXP_SUBSTR(file_url, '[^/]+$'), '%2e', '.') as filename
  FROM DIRECTORY('@avalanche_db.avalanche_schema.customer_reviews')
  WHERE filename LIKE '%.docx'
)
SELECT 
  filename,
  SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
    @avalanche_db.avalanche_schema.customer_reviews,
    filename,
    {'mode': 'layout'}
  ):content AS layout
FROM files;

### `SENTIMENT`

It accepts a given English-language input text and returns an overall sentiment score in the range of -1 to 1.

Learn more about [SENTIMENT](https://docs.snowflake.com/en/sql-reference/functions/sentiment-snowflake-cortex)

In [ ]:
SELECT SNOWFLAKE.CORTEX.SENTIMENT('The snow goggles have been a great investment for winter sports')

In [ ]:
SELECT SNOWFLAKE.CORTEX.SENTIMENT('The snow goggles were okay. ')

In [ ]:
SELECT SNOWFLAKE.CORTEX.SENTIMENT('The snow goggles was not worth the money')

In [ ]:
SELECT SNOWFLAKE.CORTEX.SENTIMENT('I would never buy the snow goggles again!')

### `SUMMARIZE`

Summarizes the given English-language input text.

Learn more about [SUMMARIZE](https://docs.snowflake.com/en/sql-reference/functions/summarize-snowflake-cortex)

In [ ]:
SELECT SNOWFLAKE.CORTEX.SUMMARIZE(
    'A critical aspect of any backpack, especially an avalanche safety pack where stability and quick access to controls are vital, is its ability to fit the user properly. We evaluated the fit and adjustability range of this specific Avalanche Safety Pack across two significantly different "users: myself at 6 2 with a longer torso, and my partner at 5 5 with a shorter torso. The results regarding its versatility were quite positive. The pack incorporates an effective torso length adjustment system, which allowed both of us to position the pack correctly so the padded hip belt rested properly on our iliac crests, bearing the majority of the load. Once the torso length was set, the shoulder straps offered ample adjustment range, and the load lifter straps effectively pulled the top of the pack closer to the shoulders, enhancing stability and comfort for both our builds. When properly cinched down, the pack remained impressively stable during simulated skiing movements and did not sway or shift excessively, which is crucial for maintaining balance and control.'
    ) AS RESPONSE

### `TRANSLATE`

Translates the given input text from one supported language to another.

Learn more about [TRANSLATE](https://docs.snowflake.com/en/sql-reference/functions/translate-snowflake-cortex)

In [ ]:
SELECT SNOWFLAKE.CORTEX.TRANSLATE(
    'A critical aspect of any backpack, especially an avalanche safety pack where stability and quick access to controls are vital, is its ability to fit the user properly. We evaluated the fit and adjustability range of this specific Avalanche Safety Pack across two significantly different "users: myself at 6 2 with a longer torso, and my partner at 5 5 with a shorter torso. The results regarding its versatility were quite positive. The pack incorporates an effective torso length adjustment system, which allowed both of us to position the pack correctly so the padded hip belt rested properly on our iliac crests, bearing the majority of the load. Once the torso length was set, the shoulder straps offered ample adjustment range, and the load lifter straps effectively pulled the top of the pack closer to the shoulders, enhancing stability and comfort for both our builds. When properly cinched down, the pack remained impressively stable during simulated skiing movements and did not sway or shift excessively, which is crucial for maintaining balance and control.',
    'en', 'de'
    ) AS RESPONSE

## Resources

- [Snowflake Cortex AI](https://www.snowflake.com/en/product/features/cortex/)
- [Snowflake Documentation](https://docs.snowflake.com/)
- [Large Language Model (LLM) Functions (Snowflake Cortex)](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions)
- Read SQL reference on [COMPLETE](https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex), [PARSE_DOCUMENT](https://docs.snowflake.com/en/sql-reference/functions/parse_document-snowflake-cortex), [SENTIMENT](https://docs.snowflake.com/en/sql-reference/functions/sentiment-snowflake-cortex), [SUMMARIZE](https://docs.snowflake.com/en/sql-reference/functions/summarize-snowflake-cortex) and [TRANSLATE](https://docs.snowflake.com/en/sql-reference/functions/translate-snowflake-cortex)